In [ ]:
import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory("/content/gdrive/MyDrive/xray",
                                                      validation_split=0.2,
                                                      subset="training",   
                                                      seed=123,
                                                      image_size=(img_height, img_width),
                                                      batch_size=batch_size
                                                        )

val_ds = tf.keras.utils.image_dataset_from_directory("/content/gdrive/MyDrive/xray",
                                                      validation_split=0.2,
                                                      subset="validation",     
                                                      seed=123,
                                                      image_size=(img_height, img_width),
                                                      batch_size=batch_size
                                                        )


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

test_ds = val_ds.take(10) 
val_ds = val_ds.skip(10)

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# Gender Classification on TPU

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
IMAGE_SIZE = [180, 180]
#CLASS_NAMES = ["NORMAL", "PNEUMONIA"]

In [ ]:
## Build the CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def conv_block(filters, inputs):
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dropout(dropout_rate)(x)

    return outputs

In [ ]:
def build_model():
    inputs = keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
## Train the model

In [ ]:
### Defining callbacks

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
initial_learning_rate = 0.015
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

In [ ]:
### Fit the model

In [ ]:
with strategy.scope():
    model = build_model()

    METRICS = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="binary_crossentropy",
        metrics=METRICS,
    )

history = model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds,
    #class_weight=class_weight,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
## Visualizing model performance

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(["precision", "recall", "binary_accuracy", "loss"]):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history[met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
### Predict and evaluate results

In [ ]:
model.evaluate(test_ds, return_dict=True)

In [ ]:
for image, label in test_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
# imageId - same as the PNG filename
# gender - 0 for female and 1 for male
CLASS_NAMES = ["FEMALE", "MALE"]

In [ ]:
for image, label in test_ds.take(1):
  for j in range(0,30):
    prediction = model.predict(test_ds.take(1))[j]
    scores = [1 - prediction, prediction]
    for score, name in zip(scores, CLASS_NAMES):
        print("This image is %.2f percent %s" % ((100 * score), name))
    plt.imshow(image[j] / 255.0)
    plt.title(CLASS_NAMES[label[j].numpy()])
    plt.show()